# Vehicle

[Link](https://vehicledatabases.com/)

In [12]:
import requests 

api_key="ed827b6218834b5b99cc644e6d5ccb7e"

In [14]:
import pandas as pd
import json

In [15]:
start_year = 2019
end_year = 2024
vehicle_year_make = []

def get_vehicle_make_year(year):
    url = "https://api.vehicledatabases.com/market-value/options/v3/make/"+ str(year)
    payload={}
    headers = {
        'x-AuthKey': api_key
    }
    result = requests.request("GET", url, headers=headers, data=payload)
    return result.text

for year in range(start_year, end_year):  # range's upper limit is exclusive, so use 2025
    result_text = get_vehicle_make_year(year)
    result = json.loads(result_text)
    year_data = str(result["data"]["year"])
    makes = result["data"]["makes"]
    for make in makes:
        vehicle_year_make.append({"Year": year_data, "Make": make})

vehicle_year_make_df = pd.DataFrame(vehicle_year_make)
print("DONE")

DONE


In [16]:
vehicle_year_make_df

,Year,Make
0,2019,Acura
1,2019,Alfa Romeo
2,2019,Aston Martin
3,2019,Audi
4,2019,Bentley
...,...,...
206,2023,Subaru
207,2023,Tesla
208,2023,Toyota
209,2023,Volkswagen


In [11]:
vehicle_year_make_model = []

def get_vehicle_model_make_year(year, make):
    url = "https://api.vehicledatabases.com/market-value/options/v3/model/" + str(year) + "/" + make
    payload={}
    headers = {
        'x-AuthKey': api_key
    }
    result = requests.request("GET", url, headers=headers, data=payload)
    return result.text

for index, row in vehicle_year_make_df.iterrows():
    year = row["Year"]
    make = row["Make"]
    
    result_text = get_vehicle_model_make_year(year, make)
    result = json.loads(result_text)
    models = result["data"]["models"]
    for model in models:
        vehicle_year_make_model.append({
            "Year": year
            , "Make": make
            , "Model": model
        })

vehicle_year_make_model_df = pd.DataFrame(vehicle_year_make_model)
print("DONE")

DONE


In [ ]:
vehicle_year_make_model_df

In [ ]:
import duckdb

con = duckdb.connect("vehicle_database.db")
results = con.sql("select distinct Year, Make from vehicle_year_make_model").df()



In [ ]:
vehicle_year_make_model_electric = []

def get_vehicle_model_make_year_electric(year, make):
    url = "https://api.vehicledatabases.com/electric-vehicle/options/v2/model/" + str(year) + "/" + make
    payload={}
    headers = {
        'x-AuthKey': api_key
    }
    result = requests.request("GET", url, headers=headers, data=payload)
    return result.text

for index, row in new_vehicle_year_make_model_df.iterrows():
    year = row["Year"]
    make = row["Make"]
    
    result_text = get_vehicle_model_make_year_electric(year, make)
    result = json.loads(result_text)
    if "data" in result and len(result["data"]) > 0:
        models = result["data"]
        print(models)
        for model in models:
            print(model)
            vehicle_year_make_model_electric.append({
                "Year": year
                , "Make": make
                , "Model": model
            })
    else:
        continue

vehicle_year_make_model_electric_df = pd.DataFrame(vehicle_year_make_model_electric)
print("done")

In [ ]:
vehicle_year_make_model_electric_df[["Make", "Model"]].drop_duplicates()

In [ ]:
pip install jupysql pandas matplotlib duckdb-engine

In [ ]:
pip install duckdb

In [ ]:
pip install dbt-core

In [ ]:
pip install dbt-core dbt-duckdb

In [17]:
import duckdb

with duckdb.connect("vehicle_database_dev.db") as con:
    results = con.sql("select distinct Year, Make from vehicle_year_make_model").df()

In [18]:
results

,Year,Make
0,2019,BMW
1,2019,Honda
2,2019,Hyundai
3,2019,Land Rover
4,2019,Porsche
...,...,...
206,2023,Dodge
207,2023,Infiniti
208,2023,Jaguar
209,2023,Lucid


In [34]:
vehicle_year_make_model_market_value = []
counter = 0

con = duckdb.connect("vehicle_database.db")
results = con.sql("select distinct Year, Make, Model from vehicle_year_make_model where Year != 2019 or (Make not in ('Audi','Alfa Romeo','Acura','Aston Martin') and Model not in ('RDX','RLX','TLX','MDX','Stelvio','Db11','A4 Allroad','NSX','Giulia','ILX','Vantage'))").df()

def get_vehicle_year_make_model_market_value(year, make, model):
    url = "https://api.vehicledatabases.com/market-value/v2/ymm/" + str(year) + "/" + make + "/" + model
    payload={}
    headers = {
        'x-AuthKey': api_key
    }
    result = requests.request("GET", url, headers=headers, data=payload)
    return result.text

for index, row in results.iterrows():
    year = row["Year"]
    make = row["Make"]
    model = row["Model"]
    
    result_text = get_vehicle_year_make_model_market_value(year, make, model)
    result = json.loads(result_text)

    if "data" in result:
        market_value_datas = result["data"]["market_value"]["market_value_data"]

        counter = counter + 1
        print("counter " + str(counter) + ": " + str(year) + " | " + make + " | " + model)

        for market_value_data in market_value_datas:
            trim = market_value_data["trim"]
            market_value_details = market_value_data["market value"]

            for market_value_detail in market_value_details:
                condition = market_value_detail["Condition"]
                trade_in = market_value_detail["Trade-In"]
                private_party = market_value_detail["Private Party"]
                dealer_detail = market_value_detail["Dealer Retail"]

                print(trim + " | " + condition + " | " + trade_in + " | " + private_party + " | " + dealer_detail)

                vehicle_year_make_model_market_value.append({
                    "year": year
                    , "make": make
                    , "model": model
                    , "trim": trim
                    , "condition": condition
                    , "trade_in": trade_in
                    , "private_party": private_party
                    , "dealer_detail": dealer_detail
                })
            
            print("\n")
    else:
        continue

vehicle_year_make_model_value_df = pd.DataFrame(vehicle_year_make_model_market_value)
print("DONE")
con.close()

counter 1: 2019 | BMW | X2
M35i 4dr All-wheel Drive Sports Activity Coupe Automatic | Outstanding | $20,121 | $22,528 | $24,842
M35i 4dr All-wheel Drive Sports Activity Coupe Automatic | Clean | $19,755 | $22,116 | $24,385
M35i 4dr All-wheel Drive Sports Activity Coupe Automatic | Average | $19,144 | $21,429 | $23,623
M35i 4dr All-wheel Drive Sports Activity Coupe Automatic | Rough | $18,351 | $20,536 | $22,632


xDrive28i 4dr All-wheel Drive Sports Activity Coupe Automatic | Outstanding | $16,919 | $19,070 | $21,138
xDrive28i 4dr All-wheel Drive Sports Activity Coupe Automatic | Clean | $16,611 | $18,721 | $20,749
xDrive28i 4dr All-wheel Drive Sports Activity Coupe Automatic | Average | $16,098 | $18,140 | $20,100
xDrive28i 4dr All-wheel Drive Sports Activity Coupe Automatic | Rough | $15,431 | $17,383 | $19,257


sDrive28i 4dr Front-wheel Drive Sports Activity Coupe Automatic | Outstanding | $15,261 | $17,671 | $19,987
sDrive28i 4dr Front-wheel Drive Sports Activity Coupe Automatic |

KeyboardInterrupt: 

In [ ]:
vehicle_year_make_model_electric = []

def get_vehicle_model_make_year_electric(year, make):
    url = "https://api.vehicledatabases.com/electric-vehicle/options/v2/model/" + str(year) + "/" + make
    payload={}
    headers = {
        'x-AuthKey': api_key
    }
    result = requests.request("GET", url, headers=headers, data=payload)
    return result.text

for index, row in results.iterrows():
    year = row["Year"]
    make = row["Make"]
    
    result_text = get_vehicle_model_make_year_electric(year, make)
    result = json.loads(result_text)
    if "data" in result and len(result["data"]) > 0:
        models = result["data"]
        print(models)
        for model in models:
            vehicle_year_make_model_electric.append({
                "Year": year
                , "Make": make
                , "Model": model
            })
    else:
        continue

vehicle_year_make_model_electric_df = pd.DataFrame(vehicle_year_make_model_electric)
print("done")

In [20]:
vehicle_year_make_model_electric_df

,Year,Make,Model
0,2019,BMW,I3
1,2019,Honda,Clarity Electric
2,2019,Honda,Clarity Fuel Cell
3,2019,Hyundai,Ioniq EV
4,2019,Hyundai,Kona Electric
...,...,...,...
146,2023,Lucid,AIR
147,2023,Tesla,Model 3
148,2023,Tesla,Model S
149,2023,Tesla,Model X


In [21]:
with duckdb.connect("vehicle_database_dev.db") as con:
    con.sql("CREATE TABLE vehicle_year_make_model_electric AS SELECT * FROM vehicle_year_make_model_electric_df")

In [23]:
with duckdb.connect("vehicle_database_dev.db") as con:
    results = con.sql("select distinct Make, Model from vehicle_year_make_model_electric").df()

results

,Make,Model
0,Mazda,MX 30 EV
1,Tesla,Model S
2,BMW,IX
3,Volvo,C40 Recharge Pure Electric
4,Mercedes-Benz,EQB 350
...,...,...
60,BMW,I4
61,Rivian,R1S
62,Audi,E Tron
63,Nissan,Leaf


In [36]:
vehicle_year_make_model_value_df_2 = pd.DataFrame(vehicle_year_make_model_market_value)

In [41]:
vehicle_year_make_model_value_df
# vehicle_year_make_model_value_df_2

,year,make,model,trim,condition,trade_in,private_party,dealer_detail
0,2019,Acura,ILX,Base 4dr Sedan Automatic,Outstanding,"$15,660","$17,937","$20,132"
1,2019,Acura,ILX,Base 4dr Sedan Automatic,Clean,"$15,313","$17,527","$19,659"
2,2019,Acura,ILX,Base 4dr Sedan Automatic,Average,"$14,734","$16,845","$18,872"
3,2019,Acura,ILX,Base 4dr Sedan Automatic,Rough,"$13,981","$15,957","$17,847"
4,2019,Acura,ILX,Technology & A-SPEC Packages 4dr Sedan Automatic,Outstanding,"$18,408","$21,376","$24,239"
...,...,...,...,...,...,...,...,...
147,2019,Aston Martin,Vantage,Base 2dr Coupe Automatic,Rough,"$47,050","$48,627","$50,007"
148,2019,Audi,A4 Allroad,2.0T Premium 4dr All-wheel Drive quattro Wagon...,Outstanding,"$20,478","$24,517","$28,414"
149,2019,Audi,A4 Allroad,2.0T Premium 4dr All-wheel Drive quattro Wagon...,Clean,"$20,024","$23,957","$27,747"
150,2019,Audi,A4 Allroad,2.0T Premium 4dr All-wheel Drive quattro Wagon...,Average,"$19,267","$23,024","$26,635"


In [44]:
with duckdb.connect("vehicle_database.db") as con:
    # con.sql("DROP TABLE vehicle_year_make_model_value")
    # con.sql("CREATE TABLE vehicle_year_make_model_value AS SELECT * FROM vehicle_year_make_model_value_df")
    con.sql("INSERT INTO vehicle_year_make_model_value BY NAME SELECT * FROM vehicle_year_make_model_value_df_2")

In [10]:
import duckdb

con = duckdb.connect("vehicle_database_dev.db")
results = con.sql("select distinct hash(make || '-' || model || '-' || strftime(current_localtimestamp(), '%Y-%m-%d %H:%M:%S') || '-' || strftime(current_localtimestamp(), '%Y-%m-%d %H:%M:%S')) AS car_id, make, model, current_localtimestamp() as created_timestamp, current_localtimestamp() as updated_timestamp from vehicle_year_make_model_value").df()
con.close()

In [11]:
results

,car_id,make,model,created_timestamp,updated_timestamp
0,6291126424295784317,Chevrolet,Silverado 1500 LD,2024-11-22 17:20:22.524,2024-11-22 17:20:22.524
1,14171881846227290074,KIA,Sedona,2024-11-22 17:20:22.524,2024-11-22 17:20:22.524
2,9567078782986015187,Volvo,Xc90,2024-11-22 17:20:22.524,2024-11-22 17:20:22.524
3,11156316456754213472,BMW,X7,2024-11-22 17:20:22.524,2024-11-22 17:20:22.524
4,3714681737635820524,BMW,Z4,2024-11-22 17:20:22.524,2024-11-22 17:20:22.524
...,...,...,...,...,...
95,11212229095495652729,BMW,Alpina XB7,2024-11-22 17:20:22.524,2024-11-22 17:20:22.524
96,16090122919132570905,Chevrolet,Equinox,2024-11-22 17:20:22.524,2024-11-22 17:20:22.524
97,17009959646572705884,Lexus,GX 460,2024-11-22 17:20:22.524,2024-11-22 17:20:22.524
98,9659026801568551786,Lexus,LX 570,2024-11-22 17:20:22.524,2024-11-22 17:20:22.524
